In [ ]:
import os, glob
from IPython.display import clear_output

fpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/merra2-reanalysis"

# Download subset of data from MERRA2

Get list of files to download from https://disc.gsfc.nasa.gov/datasets?keywords=merra2&page=1

Example url: https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2SDNXSLV.5.12.4/1980/01/MERRA2_100.statD_2d_slv_Nx.19800101.nc4.nc4?T2MMAX[0:0][60:360][0:575],TPRECMAX[0:0][60:360][0:575],time,lat[60:360],lon[0:575]


In [2]:
# read the full file list
data = []
with open("allfiles_M2SDNXSLV_5.12.4_20260108_113901_.txt","r") as f:
    data = f.readlines() # readlines() returns a list of items, each item is a line in your file

In [ ]:
for y in range(1980,2026):

    print(y)
    
    # check if file has already been downloaded
    if os.path.exists(fpath+"/"+str(y)+"/MERRA2_100.statD_2d_slv_Nx."+str(y)+"1231.nc4.nc4"): continue 

    # make a new directory to hold the data
    !mkdir -p $fpath/$y

    # select only the files for the current year
    fl_y = [r for r in data if "/"+str(y)+"/" in r]
    with open('subset-merra2_tmp.txt', 'w') as f:
        for line in fl_y:
            f.write(f"{line}")
    ! cd $fpath/$y; wget --user=clairbarnes --password=Wo0M0r3D4t4! --content-disposition -i ../subset-merra2_tmp.txt

    clear_output(wait = False)
print("Done.")

In [7]:
# check all files downloaded successfully
for y in range(1980,2026):
    fl = glob.glob(fpath+"/"+str(y)+"/*.nc4")
    if len(fl) < 365: print(y, len(fl))

2025 335


### Compile into one file per variable

In [ ]:
# first compile individual years (both variables)
for y in range(1980,2026):
    
    fl = glob.glob(fpath+str(y)+"*/*.nc4")
    if y < 2025 and len(fl) < 365: continue

    print(y)

    # compile into file per single year
    y_fnm = fpath+"merra2_"+str(y)+".nc"

    if not os.path.exists(y_fnm):
        print("  Compiling...")
        ! module load CDO/2.4.4-gompi-2024a; cdo mergetime $fpath$y/*.nc4 $y_fnm

    clear_output(wait = False)
print("Done.")

Done.


In [3]:
# compile into single file per variable
dpath = "/rds/general/user/cb2714/home/99_ephemera/quick-study"
! module load CDO/2.4.4-gompi-2024a; cdo -setname,tmax -selvar,T2MMAX -mergetime $fpath/merra2_[0-9]*.nc $dpath/data/tmax_merra2.nc
! module load CDO/2.4.4-gompi-2024a; cdo -setname,pr -selvar,TPRECMAX -mergetime $fpath/merra2_[0-9]*.nc $dpath/data/pr_merra2.nc

cdo(1) selname: Process started
cdo(2) mergetime: Process started
cdo    setname: Processed 2907862272 values from 1 variable over 16772 timesteps [44.55s 109MB]
cdo(1) selname: Process started
cdo(2) mergetime: Process started
cdo    setname: Processed 2907862272 values from 1 variable over 16772 timesteps [44.75s 109MB]


# Download FWI from GFWED

No subsetting available, have to download global file each time

In [ ]:
import os
from IPython.display import clear_output

fpath = "/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/merra2-fwi"

# download all daily files from NASA
for y in [str(y) for y in range(1994,1979,-1)]:
    ! mkdir -p $fpath/$y
    for m in [str(m).rjust(2, "0") for m in range(1,13)]:
        for d in [str(d).rjust(2, "0") for d in range(1,32)]:
            
            url = "https://portal.nccs.nasa.gov/datashare/GlobalFWI/v2.0/fwiCalcs.MERRA2/Default/MERRA2.CORRECTED/"+str(y)+"/FWI.MERRA2.CORRECTED.Daily.Default."+y+m+d+".nc"
            outfile = fpath+"/"+y+"/"+url.split("/")[-1]

            if os.path.exists(outfile): continue
            ! wget $url -O $outfile

            clear_output(wait = False)
            
    # delete any files of size zero (nonexistent dates)
    ! find $fpath/$y/ -size 0 -delete
    
print("Done.")

--2026-01-12 18:18:24--  https://portal.nccs.nasa.gov/datashare/GlobalFWI/v2.0/fwiCalcs.MERRA2/Default/MERRA2.CORRECTED/1994/FWI.MERRA2.CORRECTED.Daily.Default.19940201.nc
Resolving portal.nccs.nasa.gov (portal.nccs.nasa.gov)... 2001:4d0:2418:2800::a99a:9791, 169.154.151.145
connected. to portal.nccs.nasa.gov (portal.nccs.nasa.gov)|2001:4d0:2418:2800::a99a:9791|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 4317424 (4.1M) [application/octet-stream]
Saving to: ‘/rds/general/user/cb2714/home/00_WWA_project_folder/ephemeral/merra2-fwi/1994/FWI.MERRA2.CORRECTED.Daily.Default.19940201.nc’

          /rds/gene   0%[                    ]       0  --.-KB/s               